In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

In [4]:
%matplotlib widget

In [5]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

## Discahrge

In [29]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200924/Bead1/Discharge/Recharge_20200925/', start_file=0)

70  files in folder
70  files loaded


In [11]:
bb = bdf_discharge[0]
bb.electrode_channels

array([1, 2], dtype=uint16)

In [27]:
dis_z = [np.std(bb.response_at_freq2('z', drive_freq=71)) for bb in bdf_discharge]

In [74]:
ch = [bb.electrode_channels[0] for bb in bdf_discharge]

In [29]:
_,ax = plt.subplots(figsize=(9.1,4))
ax.scatter(range(len(dis_z)), dis_z/dis_z[0], label='std')
ax.scatter(range(len(dis_z)), amps/amps[0], label='fit')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
bdf_discharge_z = [bb for bb,ch_ in zip(bdf_discharge, ch)  if ch_==1 ]
bdf_discharge_x = [bb for bb,ch_ in zip(bdf_discharge, ch)  if ch_==4 ]

In [76]:
index_z = [bb for bb,ch_ in zip(range(len(bdf_discharge)), ch)  if ch_==1 ]
index_x = [bb for bb,ch_ in zip(range(len(bdf_discharge)), ch)  if ch_==4 ]

In [71]:
m1 = gfw.build_z_response(bdf_discharge, drive_freq=71, charges=1, bandwidth=2, decimate=10, include_sigma=True, bimodal=False)

***************************************************
Z2-amplitude:  4.34e+02
reduced chi2:  9.318084811908948
***************************************************
Z2-amplitude:  4.09e+02
reduced chi2:  9.574709644861358
***************************************************
Z2-amplitude:  3.72e+02
reduced chi2:  9.824914975818828
***************************************************
Z2-amplitude:  4.44e+02
reduced chi2:  9.421113095866765
***************************************************
Z2-amplitude:  4.25e+02
reduced chi2:  9.44492630131021
***************************************************
Z2-amplitude:  3.36e+02
reduced chi2:  9.305279518412256
***************************************************
Z2-amplitude:  3.18e+02
reduced chi2:  9.293684699273042
***************************************************
Z2-amplitude:  3.31e+02
reduced chi2:  9.502907961635959
***************************************************
Z2-amplitude:  3.22e+02
reduced chi2:  9.184956380784522
********************

In [15]:
m2 = gfw.build_x_response(bdf_discharge_x, drive_freq=71, charges=1, bandwidth=1)

***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.5077204319970472
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.6561307426139402
***************************************************
X2-amplitude:  1.23e+01
reduced chi2:  1.2313121190426974
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.16464891136188
***************************************************
X2-amplitude:  1.21e+01
reduced chi2:  1.2168587862400966
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.2696742562620946
***************************************************
X2-amplitude:  1.23e+01
reduced chi2:  1.458111243801847
***************************************************
X2-amplitude:  1.23e+01
reduced chi2:  1.2708153177285613
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.4039743199327412
*************

In [77]:
dis_x = [np.std(bb.response_at_freq('x', drive_freq=71)) for bb in bdf_discharge_x]
dis_z = [np.std(bb.response_at_freq2('z', drive_freq=71)) for bb in bdf_discharge]

In [72]:
amps = np.array([m_.values[0] for m_ in m1])
phases = np.array([m_.values[3] for m_ in m1])
_, ax = plt.subplots(1,2,figsize=(9.1,4))
ax[0].scatter(range(len(amps)), amps)
ax[1].scatter(range(len(amps)), phases)
np.mean(phases[20:50])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

71.74075198338863

In [80]:
_,ax = plt.subplots(figsize=(9.5,4))
scale = 81
scale2 = 50
scaleX = -0.8
scaleX2 = 9200

# Perrinf
ax.scatter(np.arange(len(amps)), amps/scale, label='fit fixed-phase')
# ax.scatter(range(len(dis_z)), np.array(dis_z)/scale2, label='Z response')
# ax.scatter(range(len(dis_z)), np.array(dis_z)/scale2, label='Z response')
# ax.scatter(index_x, ampsx/scaleX, label='X response')
ax.scatter(index_x, np.array(dis_x)/scaleX2, label='X-pos fit fixed-phases')
ax.scatter(np.arange(len(amps)), ch, label='channel')

ax.legend()
ax.set(xlabel='dataset', ylabel='Charges', ylim=(None,None), xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## TF

In [6]:
# 7.6um German bead
fname = r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFuncX2_1Hz_300s.h5'
tf_x = BeadDataFile.BeadDataFile(fname=fname)
fname = r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFuncY1_1Hz_100s.h5'
tf_y = BeadDataFile.BeadDataFile(fname=fname)
fname = r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFuncZ2_1Hz_300s.h5'
tf_z = BeadDataFile.BeadDataFile(fname=fname)
tf_list = [tf_x, tf_y, tf_z]

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [8]:
_,ax = plt.subplots(1,3, figsize=(9,3))
bb = tf_z
ax[0].semilogy(*bb.psd2('x'))
ax[1].semilogy(*bb.psd2('y'))
ax[2].semilogy(*bb.psd2('z'))
[a.set(xlim=(0,100)) for a in ax]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[(0.0, 100.0)], [(0.0, 100.0)], [(0.0, 100.0)]]

In [50]:
names = ['Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9']
names = ['Y7', 'Y8', 'Y9', 'Y11']
fnames = [r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFunc'+y_+'_7Hz_10s.h5' for y_ in names]
bbs = [BeadDataFile.BeadDataFile(fname_) for fname_ in fnames]

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [56]:
_,ax = plt.subplots()
[ax.semilogy(*bb.psd2('y'), label=name_) for bb,name_ in zip(bbs,names)]
ax.legend()
ax.set(xlim=(100,1000))
tmp_freq, tmp_psd = bbs[-1].psd2('x')
np.sqrt(tmp_psd[tmp_freq==71]/tmp_psd[tmp_freq==50])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([2.06282349])

## Nois floor

In [30]:
bdf_z = bdf_discharge[40:45]
bdf_x = bdf_discharge[30:35]

freq = bdf_z[0].psd2('z')[0]
z_scale1 = bdf_z[0].psd2('z')[1][freq==71]
for bb in bdf_z:
    z_scale1 += bb.psd2('z')[1][freq==71]
z_scale1 /= len(bdf_z)

x_scale1 = bdf_x[0].psd2('x')[1][freq==71]
for bb in bdf_x:
    x_scale1 += bb.psd2('x')[1][freq==71]
x_scale1 /= len(bdf_x)

charges = 3
force = charges * 1.6e-19 * 20 / 8.6e-3 * 0.63 * np.sqrt(10) # in Newtons/sqrt(Hz)
z_scale1 = force/np.sqrt(z_scale1)
x_scale1 = force/np.sqrt(x_scale1)

In [32]:
del data_bdfs

In [91]:
## 100 files of shaking data for nose floor
num_files = 100
data_bdfs = load_dir(dirname='/data/new_trap/20200924/Bead1/Shaking/Shaking345/',
                         file_prefix='SpinShaking2_', start_file=0, max_file=num_files)
freq = data_bdfs[0].psd2('x')[0]
fftx1 = data_bdfs[0].psd2('x')[1]
ffty1 = data_bdfs[0].psd2('y')[1]
fftz1 = data_bdfs[0].psd2('z')[1]
for i in range(1,num_files):
    fftx1 += data_bdfs[i].psd2('x')[1]
    fftz1 += data_bdfs[i].psd2('z')[1]
    ffty1 += data_bdfs[i].psd2('y')[1]
fftx1/=num_files
ffty1/=num_files
fftz1/=num_files

157  files in folder
100  files loaded


In [92]:
_,ax = plt.subplots(1,2,figsize=(9.1,4), sharex=True)
# tf_freq = np.interp(freq, gfw.tf_freq, gfw.tf_ffts[2])
ax[0].semilogy(freq, np.sqrt(fftx)*x_scale1, label='Spin')
ax[0].semilogy(freq, np.sqrt(fftx1)*x_scale1, label='Ringdown', alpha=0.6)
ax[1].semilogy(freq, np.sqrt(fftz)*z_scale1, label='Spin')
ax[1].semilogy(freq, np.sqrt(fftz1)*z_scale1, label='Ringdown', alpha=0.6)
[a.legend() for a in ax]
ax[0].set(xlim=(1,50), ylim=(9e-18,1e-14), xlabel='Frequency [Hz.]', ylabel=r'ASD [N/$\sqrt{Hz}$]', title='X force')
_ = ax[1].set(xlim=(1,50), ylim=(9e-18,1e-14), xlabel='Frequency [Hz.]', title=r'Z force')
# print(np.sqrt(fftx1[freq==18])*x_scale/np.sqrt(10), np.sqrt(fftz1[freq==18])*z_scale/np.sqrt(10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
## 100 files of shaking data for nose floor
num_files = 100
data_bdfs = load_dir(dirname='/data/new_trap/20200924/Bead1/Shaking/Shaking340/',
                         file_prefix='NoSpinNoShaking0_', start_file=500, max_file=num_files)
bb = data_bdfs[0]
freq = np.fft.rfftfreq(len(bb.spin_data[0]), d=1./5000/40)
fft11 = np.abs(np.fft.rfft(bb.spin_data[0]))**2
for i in range(1,num_files):
    fft11 += np.abs(np.fft.rfft(data_bdfs[i].spin_data[0]))**2
fft11/=num_files

_,ax = plt.subplots()
ax.semilogy(freq, fft)
ax.semilogy(freq, fft11)
ax.set(xlim=(1,5000))

2000  files in folder
100  files loaded


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(1.0, 5000.0)]

In [87]:
def func1_height(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    return np.mean(bb.bead_height)

In [88]:
func1_height(101)

101   ******************


180.12130227424302

In [77]:
from joblib import Parallel, delayed
from tqdm import tqdm
test1_zfb = Parallel(n_jobs=20)(delayed(func1_height)(i) for i in tqdm(range(2500)))

100%|██████████| 2500/2500 [00:09<00:00, 257.42it/s]


In [79]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(2500), test1_zfb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
i = 1400
fname = r'/data/new_trap/20200921/Bead1/Discharge/HeightControl2/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)

In [18]:
f = h5py.File(fname,'r')
pos_data = np.array(f['pos_data'])
quad_data = np.array(f['quad_data'])

In [20]:
sync = pos_data.reshape(-1,11).T[5]

In [21]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.plot(range(500), sync[:500]/100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
i_start = np.nonzero(sync[:500])[0][0]
zfb = bb.feedback[2][i_start-1:]
x_sync = bb.xyz[0][i_start-1:]

In [65]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.plot(range(40000), zfb[100000:140000]/np.mean(zfb[:40000]), label='z_fb')
ax.plot(range(40000), bb.z2[100000:140000]/np.mean(bb.z2[:40000])*50-49, 'r--',label='z', alpha=(0.5))
ax.plot(range(40000), bb.Iz[100000:140000]/np.mean(bb.Iz[:40000])*50-49,label='Iz', alpha=(0.5))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
_,ax = plt.subplots(figsize=(9.2,4))
# ax.plot(range(40000), x_sync[:40000]/np.mean(x_sync[:40000]), label='z_fb')
ax.plot(range(40000), bb.z2[100000:140000]/np.mean(bb.z2[:40000]), 'r--',label='Iz', alpha=(0.5))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Scan Y

In [12]:
i = 0
fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
np.mean(bb.cant_pos[1])*50

213.35106397161607

In [20]:
def func1_scan(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    return np.mean(bb.cant_pos[1])*50

In [125]:
def func2_scan(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    freq_, fft_ = bb.psd2('x')
    return fft_[freq_==9]

In [156]:
def func3_scan(i):
    print(i,'  ******************')
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
#     freq_, fft_ = bb.psd2('x')
    x = bb.x2
    fft_ = np.fft.rfft(x,norm='ortho')
    freq_ = np.fft.rfftfreq(len(x), d=1./5000)
    return np.angle(fft_[freq_==9])

In [124]:
from joblib import Parallel, delayed
from tqdm import tqdm
test1_zfb = Parallel(n_jobs=40)(delayed(func1_scan)(i) for i in tqdm(range(4000)))

100%|██████████| 4000/4000 [02:21<00:00, 26.44it/s]


In [127]:
test2_9hz = Parallel(n_jobs=40)(delayed(func2_scan)(i) for i in tqdm(range(4000)))

100%|██████████| 4000/4000 [00:11<00:00, 351.41it/s]


In [157]:
test3_18hz = Parallel(n_jobs=12)(delayed(func3_scan)(i) for i in tqdm(range(3000)))

100%|██████████| 3000/3000 [01:52<00:00, 25.91it/s]


In [146]:
np.mean(np.array(test2_9hz).reshape(-1,10), axis=1).shape

(400,)

In [149]:
_,ax = plt.subplots(figsize=(9,4))
# ax.scatter(test1_zfb, np.sqrt(test2_3hz)*x_scale1)
# ax.scatter(test1_zfb, np.sqrt(test2_6hz)*x_scale1)
# ax.scatter(test1_zfb, np.sqrt(test22_6hz)*x_scale1)
# ax.scatter(test1_zfb, np.sqrt(test2_9hz)*x_scale1)
ax.semilogy(np.mean(np.array(test1_zfb).reshape(-1,5), axis=1), np.mean(np.sqrt(test2_9hz).reshape(-1,5), axis=1)*x_scale1, '.')
# ax.scatter(test1_zfb, np.sqrt(test22_9hz)*x_scale1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [160]:
_,ax = plt.subplots(figsize=(9,4))
# ax.scatter(test1_zfb, test3_3hz)
# ax.scatter(test1_zfb, test3_6hz)
# ax.scatter(test1_zfb, test3_9hz)
# ax.scatter(test1_zfb, test33_9hz)
ax.scatter(test1_zfb, test33_18hz, label='y scan')
ax.scatter(test1_zfb[:3000], test3_18hz, label='constant')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …